<div style="background-color: lightgreen; border-radius: 5px; padding: 10px;">
    <h4>LDa Topic Modeling</h4>
    <p>...</p>
</div>

### Imports

In [2]:
from preprocessing_functions import *
from topic_modeling_functions import *

import warnings
warnings.filterwarnings('ignore')

C:\Users\diede\Personal Files [Local]\Applied Data Science\Thesis - CITYNET\venv_citynet3\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [4]:
new_data = False

if new_data:
    INPUT_DIR = "../../../../../data/clean/city_pair_paragraphs3/"
    BATCHES = [5, 10, 20, 30, 40]
    POS = ["NOUN", "VERB", "ADJ"]
    ONLY_ENGLISH_WORDS = True
    sort_by_paragraphs_count = True
    merged_POS = True

    data_list = import_lemmatised_paragraphs(INPUT_DIR,
                                             POS,
                                             BATCHES,
                                             ONLY_ENGLISH_WORDS=ONLY_ENGLISH_WORDS,
                                             merged_POS=merged_POS,
                                             sort_by_paragraphs=sort_by_paragraphs_count)
    
    # Merge into single dataframe
    frames = [citypair['lemmatized_paragraphs'] for citypair in data_list]
    citypairs = [citypair['city_pair'] for citypair in data_list]

    result = pd.concat(frames) #, keys=citypairs)
    result.set_index('paragraph_id', inplace=True)
    result.sort_index(inplace=True)
    result.reset_index(inplace=True)
else:
    result = pd.read_csv("../../../../../data/clean/deliverable_merged_POS_1770_city_pairs.csv")

result

BATCHES: ['5', '10', '20', '30', '40']:   0%|          | 0/5 [00:00<?, ?it/s]

City Pair:   0%|          | 0/35 [00:00<?, ?it/s]

City Pair:   0%|          | 0/145 [00:00<?, ?it/s]

City Pair:   0%|          | 0/244 [00:00<?, ?it/s]

City Pair:   0%|          | 0/306 [00:00<?, ?it/s]

City Pair:   0%|          | 0/10 [00:00<?, ?it/s]

,paragraph_id,city_pair,paragraph,merged_POS
0,1,berlin_milan,"after his tenure in academia, he continued to ...","[tenure, academia, month, year, travel, incide..."
1,2,berlin_milan,one of the astronomers selected for the search...,"[astronomer, search, priest, invitation, group..."
2,3,berlin_milan,there are plenty of air connections between ye...,"[plenty, air, connection, city, connection, ci..."
3,4,berlin_milan,"since 2009, 'the brandery', an urban fashion s...","[fashion, year, language, monitor, ranking, wo..."
4,5,berlin_milan,when considering the commuter belts or metropo...,"[commuter, belt, area, datum, population, orde..."
...,...,...,...,...
399234,401783,warsaw_sofia,"as of 2013, pap had nearly 500 employees and a...","[pap, employee, associate, journalist, corresp..."
399235,401784,warsaw_sofia,schiller became famous for his 1934 staging of...,"[staging, polish, theatre, present, famous]"
399236,401785,warsaw_sofia,the disqualification was eventually shortened ...,"[disqualification, championship, record, metre..."
399237,401786,warsaw_sofia,"boris schatz was born in varniai, in the kovno...","[governorate, empire, day, father, teacher, ch..."


In [62]:
 result40to60 = pd.read_csv("../../../../../data/clean/deliverable_lemmatised_paragraphs_cities40to60.csv")

In [64]:
import ast
result40to60['city_pair'] = result40to60['city_pair'].apply(ast.literal_eval)

In [65]:
result40to60.city_pair = result40to60.city_pair.apply(lambda tup: '_'.join(tup))

In [68]:
merged_POS = True
missing_POS = dict()
collected_POS = set()

df = result40to60
sub_df = df[['city_pair', 'paragraph_id', 'paragraph']]

if merged_POS:
    sub_df['merged_POS'] = [[] for _ in range(df.shape[0])]

combined_POS = None
for tag in POS:
    if ONLY_ENGLISH_WORDS:
        column_name = f'{tag}_clean'   
    else:
        column_name = f'{tag}'

    if column_name not in df.columns:
        if not column_name in missing_POS.keys():
            missing_POS[column_name] = []

        missing_POS[column_name].append(CITY_PAIR)

    else:
        string_to_list = df[column_name].apply(literal_eval)

        if merged_POS:
            sub_df['merged_POS'] += string_to_list    

        else:   
            sub_df[tag] = string_to_list

        collected_POS.add(tag)

if len(missing_POS):
    print(f'The following POS tags are missing: {missing_POS}')

    
# citypair_dict = {'batch': batch, 'city_pair': CITY_PAIR, 'paragraphs_count': len(df), 'english_words': ONLY_ENGLISH_WORDS, 'collected_POS': collected_POS, 'lemmatized_paragraphs': sub_df}

In [156]:
paragraphs_of_1770_city_pairs = pd.concat([result, sub_df], ignore_index=True, verify_integrity=True)

In [166]:
paragraphs_of_1770_city_pairs = paragraphs_of_1770_city_pairs.sort_index()
paragraphs_of_1770_city_pairs['indexer'] = paragraphs_of_1770_city_pairs.index

In [167]:
paragraphs_of_1770_city_pairs.to_csv("../../../../../data/clean/deliverable_merged_POS_1770_city_pairs.csv", index=False)

In [73]:
paragraphs_of_1770_city_pairs['city_pair'].nunique()

1639

### Vectorize Documents

In [168]:
paragraphs = paragraphs_of_1770_city_pairs.merged_POS
MIN_DF = 0.05
MAX_DF = 0.9

dictionary, corpus = vectorize(paragraphs,
                               MIN_DF=MIN_DF,
                               MAX_DF=MAX_DF)

### Train Single LDA Model

In [177]:
N_TOPICS = 5
model_dict2 = train_lda_model(lemmatized_text=paragraphs,
                            dictionary=dictionary,
                            corpus=corpus,
                            MIN_DF=MIN_DF,
                            MAX_DF=MAX_DF,
                            N_TOPICS=N_TOPICS
                            # random_seed=0, should be added to allow the same results!
                            )

print(model_dict.keys())

topics: 5, MIN_DF: 0, MAX_DF: 1.0
dict_keys(['lda_model', 'coherence_score', 'dictionary', 'corpus'])


### Load LDA Model

In [212]:
path = 'lda_model_6_0min_1_0max_1770_city_pairs_merged_POS'
forced_path = os.path.abspath(path)

LOAD_VIS=True,
LOAD_DICT=True,
LOAD_TEXTS=True,
LOAD_COHERENCE_SCORE=True

if os.path.exists(forced_path):
    model_dict = load_lda_model(forced_path,
                           LOAD_VIS=LOAD_VIS,
                           LOAD_DICT=LOAD_DICT,
                           LOAD_TEXTS=LOAD_TEXTS,
                           LOAD_COHERENCE_SCORE=LOAD_COHERENCE_SCORE)
    print(model_dict.keys())
else:
    print('Path is invalid')

dict_keys(['lda_model', 'coherence_score', 'visualisation', 'texts', 'dictionary'])


In [213]:
model_dict['coherence_score']

'0.6451151522688326'

### Visualisation

In [189]:
def visualise_topics(lda_model, corpus, dictionary, sort_topics=False):
    lda_conv = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_model)
    vis = gensimvis.prepare(lda_conv, corpus, dictionary, sort_topics=False)
    
    print(vis.topic_order)
    
    return vis

In [218]:
pyLDAvis.enable_notebook()

if 'visualisation' not in model_dict.keys():
    vis = visualise_topics(model_dict['lda_model'], corpus, dictionary)
else:
    vis = model_dict['visualisation']

vis

In [182]:
model_dict_old = model_dict

In [183]:
model_dict = model_dict2

### Save LDA Model

In [191]:
import os

OUTPUT_DIR = os.getcwd()
MODEL_NAME = 'lda_model_5_0min_1_0max_1770_city_pairs_merged_POS'
MODEL = model_dict['lda_model']
COHERENCE_SCORE = model_dict['coherence_score']
TEXTS = paragraphs
DICTIONARY = model_dict['dictionary']
CORPUS = model_dict['corpus']
VIS = vis

SAVE_VIS = True
SAVE_DICT = True
SAVE_TEXTS = True
SAVE_COHERENCE_SCORE = True

save_lda_model(MODEL=model_dict['lda_model'],
                OUTPUT_DIR=OUTPUT_DIR,
                NAME=MODEL_NAME,
                COHERENCE_SCORE=COHERENCE_SCORE,
                DICTIONARY=DICTIONARY,
                CORPUS=CORPUS,
                TEXTS=TEXTS,
                VIS=VIS,
                SAVE_VIS=SAVE_VIS,
                SAVE_DICT=SAVE_DICT,
                SAVE_TEXTS=SAVE_TEXTS,
                SAVE_COHERENCE_SCORE=SAVE_COHERENCE_SCORE
              )

Saving lda model...
Model has been saved to the following location: C:\Users\diede\Personal Files [Local]\Applied Data Science\Thesis - CITYNET\citynet\src\analyses\Diederik\Clean Code\lda_model_5_0min_1_0max_1770_city_pairs_merged_POS.


### Get most relevant words per topic  
##### (used to create the topic vectors in the word embedding classification method)

In [267]:
all_topics = {}
num_terms = 50 # Adjust number of words to represent each topic
lambd = 0.15 # Adjust this accordingly based on tuning above

if not isinstance(vis, pyLDAvis._prepare.PreparedData):
    vis = visualise_topics(model_dict['lda_model'], corpus, dictionary)
    
for i in range(1,len(model_dict['lda_model'].get_topics())+1): #Adjust this to reflect number of topics chosen for final LDA model
    topic = vis.topic_info[vis.topic_info.Category == 'Topic'+str(i)].copy()
    topic['relevance'] = topic['loglift']*(1-lambd)+topic['logprob']*lambd
    all_topics['Topic '+str(i)] = topic.sort_values(by='relevance', ascending=False).Term[:num_terms].values

In [268]:
relevant_words_df =  pd.DataFrame(all_topics).T
relevant_words_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
Topic 1,army,embassy,protest,police,arrest,diplomatic,treaty,mission,war,attack,...,battalion,brigade,general,foreign,constituency,escort,infantry,terrorist,political,gestapo
Topic 2,railway,route,line,station,operate,service,airport,airline,flight,passenger,...,air,build,timetable,operation,system,hourly,gauge,electrify,commuter,electrification
Topic 3,university,professor,art,research,degree,science,museum,graduate,gallery,doctorate,...,institution,student,philology,postgraduate,society,undergraduate,biology,academy,modern,physiology
Topic 4,paint,painting,painter,portrait,father,marry,family,daughter,style,fashion,...,vogue,nude,altarpiece,depiction,altar,collection,tomb,paternal,motif,codex
Topic 5,win,match,goal,team,final,game,club,score,season,league,...,midfielder,scoring,knockout,scorer,teammate,playoff,unbeaten,brace,competition,play
Topic 6,opera,festival,perform,orchestra,music,concert,film,symphony,sing,premiere,...,vocal,audition,tenor,release,quartet,orchestral,comique,operetta,stage,guitarist


### Renaming Topics

In [269]:
new_names = {topic: None for topic in relevant_words_df.index}

for topic in relevant_words_df.index:
    new_name = input(f"Rename '{topic}' to: ").lower()
    new_names[topic] = new_name
    # print(f'\t--> {topic}: {new_name}')

print(new_names)

saving = input("\nContinue with these names? type 'y' or 'n'.")
if saving.lower() == 'y':
    relevant_words_df_changed_names = relevant_words_df.rename(index=new_names)
    print('\tChanged topic names!')
else:
    print("\tKept old topic names!")

Rename 'Topic 1' to: diplomacy
Rename 'Topic 2' to: transportation
Rename 'Topic 3' to: education
Rename 'Topic 4' to: art
Rename 'Topic 5' to: sport
Rename 'Topic 6' to: entertainment
{'Topic 1': 'diplomacy', 'Topic 2': 'transportation', 'Topic 3': 'education', 'Topic 4': 'art', 'Topic 5': 'sport', 'Topic 6': 'entertainment'}

Continue with these names? type 'y' or 'n'.y
	Changed topic names!


In [276]:
reversed_df = relevant_words_df_changed_names.T

##### Save Relevant Words-Topic  Dataframe

In [273]:
filename = 'most_relevant_50_words_per_topic_1770_city_pairs_6_topics'

In [277]:
reversed_df.to_csv(f"..\..\..\..\..\data\clean\\{filename}.csv", index=False)

### Word-Topic Distribution

In [225]:
N_TOPICS = len(model_dict['lda_model'].get_topics())
MAX_WORDS = 8

for i in range(N_TOPICS):
    topic_words = model_dict['lda_model'].show_topic(i, topn=MAX_WORDS)
    print(relevant_words_df_changed_names.index[i], [(x[0], round(x[1], 3)) for x in topic_words])
    print()

diplomacy [('war', 0.008), ('german', 0.007), ('city', 0.007), ('year', 0.006), ('government', 0.005), ('time', 0.005), ('state', 0.005), ('return', 0.005)]

transportation [('service', 0.018), ('city', 0.015), ('line', 0.015), ('train', 0.012), ('railway', 0.012), ('station', 0.011), ('route', 0.011), ('company', 0.011)]

education [('art', 0.036), ('study', 0.028), ('work', 0.024), ('exhibition', 0.013), ('school', 0.013), ('university', 0.013), ('professor', 0.011), ('museum', 0.011)]

art [('work', 0.018), ('year', 0.011), ('bear', 0.008), ('time', 0.007), ('painting', 0.007), ('study', 0.007), ('family', 0.007), ('move', 0.007)]

sport [('win', 0.036), ('season', 0.023), ('final', 0.021), ('club', 0.02), ('team', 0.018), ('play', 0.016), ('goal', 0.016), ('match', 0.016)]

entertainment [('opera', 0.031), ('festival', 0.023), ('perform', 0.021), ('music', 0.017), ('include', 0.015), ('tour', 0.015), ('orchestra', 0.015), ('film', 0.015)]



###  Document-Topic Distribution

In [226]:
import pandas as pd

transformed_docs = model_dict['lda_model'].load_document_topics()
topic_distributions = pd.DataFrame([[x[1] for x in doc] for doc in transformed_docs], 
             columns=[f"lda_{relevant_words_df_changed_names.index[i]}" for i in range(N_TOPICS)])

In [227]:
topic_distributions
# paragraphs_of_1770_city_pairs.iloc[0]['paragraph']

,lda_diplomacy,lda_transportation,lda_education,lda_art,lda_sport,lda_entertainment
0,0.646392,0.001234,0.001288,0.324088,0.000847,0.026151
1,0.000920,0.018644,0.000922,0.977949,0.000606,0.000958
2,0.001468,0.992814,0.001471,0.001752,0.000967,0.001529
3,0.002832,0.439793,0.002837,0.549726,0.001865,0.002948
4,0.004585,0.889086,0.004593,0.005473,0.003019,0.093243
...,...,...,...,...,...,...
506297,0.001898,0.990708,0.001902,0.002266,0.001250,0.001976
506298,0.299221,0.657007,0.001032,0.001230,0.040437,0.001073
506299,0.024290,0.001148,0.532335,0.370914,0.000788,0.070525
506300,0.941534,0.011555,0.012065,0.014376,0.007930,0.012540


In [ ]:
# # Give Topics sensible names
# topic_distributions_renamed_cols=topic_distributions.rename(columns = { 'topic_1': 'lda_sport',
#                                         'topic_2': 'lda_art',
#                                         'topic_3': 'lda_diplomacy',
#                                         'topic_4': 'lda_entertainment',
#                                         'topic_5': 'lda_transportation',
#                                         'topic_6': 'lda_education'}, inplace = False)

# topic_distributions_renamed_cols.head(2)

### Topic Distributions

In [228]:
topic_distributions.idxmax(axis="columns").value_counts()

lda_entertainment     103021
lda_art                95279
lda_transportation     82653
lda_education          82578
lda_diplomacy          76192
lda_sport              66579
dtype: int64

In [229]:
topic_distributions.idxmax(axis="columns").value_counts(normalize=True)

lda_entertainment     0.203477
lda_art               0.188186
lda_transportation    0.163248
lda_education         0.163100
lda_diplomacy         0.150487
lda_sport             0.131501
dtype: float64

### Concat Topic Distributions and Documents Dataframe

In [230]:
updated_results = pd.concat([paragraphs_of_1770_city_pairs, topic_distributions],
                  axis = 1)

In [231]:
 updated_results.head(2)

,paragraph_id,city_pair,paragraph,merged_POS,indexer,lda_diplomacy,lda_transportation,lda_education,lda_art,lda_sport,lda_entertainment
0,1,berlin_milan,"after his tenure in academia, he continued to ...","[tenure, academia, month, year, travel, incide...",0,0.646392,0.001234,0.001288,0.324088,0.000847,0.026151
1,2,berlin_milan,one of the astronomers selected for the search...,"[astronomer, search, priest, invitation, group...",1,0.000920,0.018644,0.000922,0.977949,0.000606,0.000958


In [233]:
updated_results.to_csv('tempsave.csv', index=False)

### Get dominant topic and score of chunked dataframe

In [234]:
import numpy as np
nr_of_chunks = (len(updated_results) // 10000) + 1

chunked_dataframe = np.array_split(updated_results, nr_of_chunks)

In [236]:
# This may take a while

from tqdm.notebook import tqdm

for i, subdataframe in enumerate(tqdm(chunked_dataframe)):
    chunked_dataframe[i] = pd.concat([subdataframe, subdataframe[topic_distributions.columns].agg(['idxmax','max'],axis=1)], axis=1)

  0%|          | 0/51 [00:00<?, ?it/s]

In [239]:
final_df = pd.concat(chunked_dataframe,axis=0)

In [240]:
final_df

,paragraph_id,city_pair,paragraph,merged_POS,indexer,lda_diplomacy,lda_transportation,lda_education,lda_art,lda_sport,lda_entertainment,idxmax,max
0,1,berlin_milan,"after his tenure in academia, he continued to ...","[tenure, academia, month, year, travel, incide...",0,0.646392,0.001234,0.001288,0.324088,0.000847,0.026151,lda_diplomacy,0.646392
1,2,berlin_milan,one of the astronomers selected for the search...,"[astronomer, search, priest, invitation, group...",1,0.000920,0.018644,0.000922,0.977949,0.000606,0.000958,lda_art,0.977949
2,3,berlin_milan,there are plenty of air connections between ye...,"[plenty, air, connection, city, connection, ci...",2,0.001468,0.992814,0.001471,0.001752,0.000967,0.001529,lda_transportation,0.992814
3,4,berlin_milan,"since 2009, 'the brandery', an urban fashion s...","[fashion, year, language, monitor, ranking, wo...",3,0.002832,0.439793,0.002837,0.549726,0.001865,0.002948,lda_art,0.549726
4,5,berlin_milan,when considering the commuter belts or metropo...,"[commuter, belt, area, datum, population, orde...",4,0.004585,0.889086,0.004593,0.005473,0.003019,0.093243,lda_transportation,0.889086
...,...,...,...,...,...,...,...,...,...,...,...,...,...
506297,32,paris_marseille,"Since 1849, the Paris-Marseille railway passes...","[railway, northeast, station, line, rail, link...",506297,0.001898,0.990708,0.001902,0.002266,0.001250,0.001976,lda_transportation,0.990708
506298,72,hamburg_bremen,"After the war, direct trade was minimal. What ...","[war, trade, port, city, tobacco, rice, cotton...",506298,0.299221,0.657007,0.001032,0.001230,0.040437,0.001073,lda_transportation,0.657007
506299,301,london_zürich,At Vassar College Mann developed lifelong frie...,"[friendship, student, role, history, psycholog...",506299,0.024290,0.001148,0.532335,0.370914,0.000788,0.070525,lda_education,0.532335
506300,84,munich_nuremberg,"Promoted to SS-'Oberführer', Scheel on 25 Apri...","[promote, base, transfer, encompass]",506300,0.941534,0.011555,0.012065,0.014376,0.007930,0.012540,lda_diplomacy,0.941534


##### Rename Columns

In [241]:
final_df_renamed_cols = final_df.rename(columns = { 'idxmax': 'lda_dominant',
                                        'max': 'lda_dominant_score'}, inplace = False)

In [242]:
final_df_renamed_cols.head(2)

,paragraph_id,city_pair,paragraph,merged_POS,indexer,lda_diplomacy,lda_transportation,lda_education,lda_art,lda_sport,lda_entertainment,lda_dominant,lda_dominant_score
0,1,berlin_milan,"after his tenure in academia, he continued to ...","[tenure, academia, month, year, travel, incide...",0,0.646392,0.001234,0.001288,0.324088,0.000847,0.026151,lda_diplomacy,0.646392
1,2,berlin_milan,one of the astronomers selected for the search...,"[astronomer, search, priest, invitation, group...",1,0.000920,0.018644,0.000922,0.977949,0.000606,0.000958,lda_art,0.977949


### Save Document-Topic Distributions

In [264]:
final_df_renamed_cols.to_csv("..\..\..\..\..\data\clean\\final_lda_classification_1770_city_pairs_6_topics.csv", index=False)

In [260]:
final_df_renamed_cols['index_id'] = final_df_renamed_cols.index

In [262]:
display(final_df_renamed_cols)

,paragraph_id,city_pair,paragraph,merged_POS,indexer,lda_diplomacy,lda_transportation,lda_education,lda_art,lda_sport,lda_entertainment,lda_dominant,lda_dominant_score,index_id
0,1,berlin_milan,"after his tenure in academia, he continued to ...","[tenure, academia, month, year, travel, incide...",0,0.646392,0.001234,0.001288,0.324088,0.000847,0.026151,lda_diplomacy,0.646392,0
1,2,berlin_milan,one of the astronomers selected for the search...,"[astronomer, search, priest, invitation, group...",1,0.000920,0.018644,0.000922,0.977949,0.000606,0.000958,lda_art,0.977949,1
2,3,berlin_milan,there are plenty of air connections between ye...,"[plenty, air, connection, city, connection, ci...",2,0.001468,0.992814,0.001471,0.001752,0.000967,0.001529,lda_transportation,0.992814,2
3,4,berlin_milan,"since 2009, 'the brandery', an urban fashion s...","[fashion, year, language, monitor, ranking, wo...",3,0.002832,0.439793,0.002837,0.549726,0.001865,0.002948,lda_art,0.549726,3
4,5,berlin_milan,when considering the commuter belts or metropo...,"[commuter, belt, area, datum, population, orde...",4,0.004585,0.889086,0.004593,0.005473,0.003019,0.093243,lda_transportation,0.889086,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506297,32,paris_marseille,"Since 1849, the Paris-Marseille railway passes...","[railway, northeast, station, line, rail, link...",506297,0.001898,0.990708,0.001902,0.002266,0.001250,0.001976,lda_transportation,0.990708,506297
506298,72,hamburg_bremen,"After the war, direct trade was minimal. What ...","[war, trade, port, city, tobacco, rice, cotton...",506298,0.299221,0.657007,0.001032,0.001230,0.040437,0.001073,lda_transportation,0.657007,506298
506299,301,london_zürich,At Vassar College Mann developed lifelong frie...,"[friendship, student, role, history, psycholog...",506299,0.024290,0.001148,0.532335,0.370914,0.000788,0.070525,lda_education,0.532335,506299
506300,84,munich_nuremberg,"Promoted to SS-'Oberführer', Scheel on 25 Apri...","[promote, base, transfer, encompass]",506300,0.941534,0.011555,0.012065,0.014376,0.007930,0.012540,lda_diplomacy,0.941534,506300


In [263]:
final_df_renamed_cols['city_pair'].nunique()

1639